### RAG_valid -> Reflection -> final VCP (SOUP data)

In [11]:
%pip install -q openai pandas scikit-learn tqdm


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ================== Imports and OpenAI client helper ==================
import os, re, csv
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from openai import OpenAI
import getpass
from typing import Optional


def init_openai_client(api_key: Optional[str] = None, persist_env: bool = False) -> Optional[OpenAI]:
    """Initialize an OpenAI client securely.

    Behavior:
    - If api_key is provided, use it to create the client (but avoid hardcoding keys in notebooks).
    - Otherwise try the OPENAI_API_KEY environment variable.
    - If still missing, prompt the user (hidden input) to paste the key for the current session.
    - If persist_env=True, the key will be stored in os.environ for this process only (not written to disk).

    Returns an OpenAI client instance or None if no key was provided/valid.
    """
    if api_key is None:
        api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        # Secure prompt (hidden) so the key isn't visible in notebook outputs or history
        try:
            typed = getpass.getpass("Enter your OpenAI API key (input hidden). Press Enter to skip: ")
        except Exception:
            typed = None
        if typed:
            api_key = typed.strip()

    if not api_key:
        print("No OpenAI API key provided. To run code that calls OpenAI, set the OPENAI_API_KEY environment variable or call init_openai_client(api_key=...) in the notebook (not recommended to paste keys in files).")
        return None

    if persist_env:
        os.environ["OPENAI_API_KEY"] = api_key

    try:
        client = OpenAI(api_key=api_key)
        print("OpenAI client initialized.")
        return client
    except Exception as e:
        # Fail gracefully and show a short hint
        print(f"Failed to initialize OpenAI client: {e}")
        print("Check the key or your network. Don't paste long keys into the notebook permanently.")
        return None

client = init_openai_client(api_key=os.getenv(''))



No OpenAI API key provided. To run code that calls OpenAI, set the OPENAI_API_KEY environment variable or call init_openai_client(api_key=...) in the notebook (not recommended to paste keys in files).


In [ ]:
DATA_CSV = ""       # must have: code_before, code_after, comment, gt_label
OUT_CSV  = ""

MODEL_NAME = "gpt-4o"      # latest multimodal GPT-4 model (text + reasoning)
MAX_TOKENS = 6             # only need "YES" or "NO"


In [66]:
STRICT_PROMPT_HEADER = """[Context Section]

You are analyzing examples from Stack Overflow where a code snippet (code_before) was later edited (code_after) and accompanied by a comment (comment).
The task is to decide if the code changes were likely caused or motivated by the comment — that is, whether the comment influenced the edit.
The answer must be "YES" if the comment and the code edit are semantically or functionally related (e.g., the edit fixes, extends, or corrects something mentioned or implied by the comment).
Otherwise, the answer must be "NO".

[Conditions / Decision Rules]

You must answer YES only if all of the following are true:

The comment refers to appreciation first then suggests a change that addresses the issue that was solved by the edit.

There is a semantic or lexical overlap between what the comment discusses and what changed in the code.

The comment appears before or in a way that could trigger the change (not an approval after the fix).

Answer NO if any of the following are true:

The comment is off-topic, descriptive, or acknowledgment-only (“thanks,” “works now,” “perfectly correct,” etc.).

The comment’s topic is different from what changed in the code.

The edit fixes an issue not mentioned or not implied by the comment.

The comment occurs after the fix or merely confirms correctness like phrase (e.g., Thanks you, it works, its perfectly fine)

[Input Format]
code_before:
<original code>

code_after:
<modified code>

comment:
<text of the comment>

[Query / Task Instruction]

Compare the code_before and code_after.
Read the comment carefully and determine whether the change between the two code versions was caused or motivated by that comment.

Provide the final answer strictly as:

YES
or
NO

No explanation, reasoning, or extra words.
"""

def build_prompt(code_before: str, code_after: str, comment: str) -> str:
    return (
        STRICT_PROMPT_HEADER
        + "\ncode_before:\n"
        + str(code_before).strip()
        + "\n\ncode_after:\n"
        + str(code_after).strip()
        + "\n\ncomment:\n"
        + str(comment).strip()
        + "\n\n"
    )


In [67]:
def normalize_yesno(text: str) -> str:
    """
    Map raw model output to strict YES/NO.
    """
    if not text:
        return "NO"
    t = text.strip()
    if t.upper().startswith("Y"):
        return "YES"
    if t.upper().startswith("N"):
        return "NO"
    # fallback
    m = re.search(r"\b(YES|NO)\b", t.upper())
    return m.group(1) if m else "NO"


def ask_gpt4o(prompt: str) -> str:
    """
    Send prompt to GPT-4o and return normalized YES/NO.
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=MAX_TOKENS,
        temperature=0.0,      # deterministic
    )
    text = response.choices[0].message.content.strip()
    return normalize_yesno(text)


In [68]:
df = pd.read_csv(DATA_CSV)
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
df.head(2)


Rows: 222
Columns: ['PostHistoryId', 'PostId', 'code_before', 'code_after', 'CommentId', 'comment', 'CommentCreationDate', 'response_time_min', 'gt_label', 'rag_predict']


PostHistoryId    PostId                                        code_before  \
0      160940523  47371021  int tmp = r; for (int i = 0; i < String.valueO...   
1      149365200  30062165  @JsonAutoDetect( fieldVisibility = Visibility....   

                                          code_after  CommentId  \
0  int tmp = 1; for (int i = 0; i < String.valueO...   81695442   
1  @JsonAutoDetect( fieldVisibility = Visibility....   48239544   

                                             comment      CommentCreationDate  \
0  But that´s the problem. I need all the numbers...  2017-11-18T21:47:19.157   
1  It's a long story that I could probably fix wi...  2015-05-05T20:03:16.320   

   response_time_min gt_label rag_predict  
0                 43  Invalid       Valid  
1            1109993  Invalid       Valid

In [69]:
df["gt_label"] = df["gt_label"].apply(lambda x: "YES" if str(x).strip().upper() == "VALID" else "NO")
df[["gt_label"]].head()

gt_label
0       NO
1       NO
2       NO
3       NO
4       NO

In [70]:
df

PostHistoryId    PostId  \
0        160940523  47371021   
1        149365200  30062165   
2        139077176  42407230   
3         16368150   7416218   
4         16021139   7258240   
..             ...       ...   
217       98381495  32257322   
218       94248821  11716017   
219       72445684  25563702   
220       10080053   4762506   
221      101331810  32993549   

                                           code_before  \
0    int tmp = r; for (int i = 0; i < String.valueO...   
1    @JsonAutoDetect( fieldVisibility = Visibility....   
2    public String getData(String val){ String dat ...   
3    final class Node implements Comparable<Node> {...   
4    import java.io.File; import java.io.FileFilter...   
..                                                 ...   
217  public void addCurrentList() throws Exception{...   
218  Logger.getLogger(rg.asteriskjava.manager.inter...   
219  private void getR...SMS(...) { tsms = new ...;...   
220  public static String removeUserFromEmail(Strin...   
221  public Map<String, IExchangeItem> createAndIni...   

                                            code_after  CommentId  \
0    int tmp = 1; for (int i = 0; i < String.valueO...   81695442   
1    @JsonAutoDetect( fieldVisibility = Visibility....   48239544   
2    public String getData(String val){ String dat ...   71962263   
3    final class Node implements Comparable<Node> {...    8962653   
4    import java.io.File; import java.io.FileFilter...    8758824   
..                                                 ...        ...   
217  public void addCurrentList() throws Exception{...   52395882   
218  Logger.getLogger(org.asteriskjava.manager.inte...   15542965   
219  private List<ReceiveFields> rows = new ArrayLi...   39922047   
220  public static String removeUserFromEmail(Strin...    5268950   
221  public Map<String, IExchangeItem> createAndIni...   53812645   

                                               comment  \
0    But that´s the problem. I need all the numbers...   
1    It's a long story that I could probably fix wi...   
2                @TruongHieu NOW ITS PERFECTLY CORRECT   
3    @medopal: Equality and other things become har...   
4    you have also more mistakes in the way you wro...   
..                                                 ...   
217  Testing `exists()` and calling `createNewFile(...   
218  it is showing  package rg.asteriskjava.manager...   
219  i can not use your method. please see this scr...   
220           will fail hard if @ is not in the string   
221  Thanks. I did not write the method but have th...   

         CommentCreationDate  response_time_min gt_label rag_predict  
0    2017-11-18T21:47:19.157                 43       NO       Valid  
1    2015-05-05T20:03:16.320            1109993       NO       Valid  
2    2017-02-23T05:37:23.203                 80       NO       Valid  
3    2011-09-14T12:28:51.590                 11       NO       Valid  
4    2011-09-01T19:13:49.527                 33       NO       Valid  
..                       ...                ...      ...         ...  
217  2015-08-27T19:34:26.777                  8      YES       Valid  
218  2012-07-30T05:56:51.537            1538048      YES       Valid  
219  2014-08-29T08:10:24.557                 41       NO       Valid  
220  2011-01-21T18:40:12.793                 20      YES       Valid  
221  2015-10-07T13:48:06.213                879       NO       Valid  

[222 rows x 10 columns]

In [71]:
preds = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    cb = str(row.get("code_before", ""))
    ca = str(row.get("code_after", ""))
    cm = str(row.get("comment", ""))
    gt = str(row.get("gt_label", "")).strip().upper()

    prompt = build_prompt(cb, ca, cm)
    pred = ask_gpt4o(prompt)

    preds.append(pred)

df["pred_label"] = preds
df.head()


100%|██████████| 222/222 [02:19<00:00,  1.59it/s]


PostHistoryId    PostId                                        code_before  \
0      160940523  47371021  int tmp = r; for (int i = 0; i < String.valueO...   
1      149365200  30062165  @JsonAutoDetect( fieldVisibility = Visibility....   
2      139077176  42407230  public String getData(String val){ String dat ...   
3       16368150   7416218  final class Node implements Comparable<Node> {...   
4       16021139   7258240  import java.io.File; import java.io.FileFilter...   

                                          code_after  CommentId  \
0  int tmp = 1; for (int i = 0; i < String.valueO...   81695442   
1  @JsonAutoDetect( fieldVisibility = Visibility....   48239544   
2  public String getData(String val){ String dat ...   71962263   
3  final class Node implements Comparable<Node> {...    8962653   
4  import java.io.File; import java.io.FileFilter...    8758824   

                                             comment      CommentCreationDate  \
0  But that´s the problem. I need all the numbers...  2017-11-18T21:47:19.157   
1  It's a long story that I could probably fix wi...  2015-05-05T20:03:16.320   
2              @TruongHieu NOW ITS PERFECTLY CORRECT  2017-02-23T05:37:23.203   
3  @medopal: Equality and other things become har...  2011-09-14T12:28:51.590   
4  you have also more mistakes in the way you wro...  2011-09-01T19:13:49.527   

   response_time_min gt_label rag_predict pred_label  
0                 43       NO       Valid         NO  
1            1109993       NO       Valid         NO  
2                 80       NO       Valid         NO  
3                 11       NO       Valid         NO  
4                 33       NO       Valid         NO

In [72]:
df["pred_label"].value_counts()

pred_label
NO     115
YES    107
Name: count, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
cm = confusion_matrix(df["gt_label"], df["pred_label"], labels=["YES", "NO"])
print("Confusion Matrix (rows: true, columns: pred):\n")
print(pd.DataFrame(cm, index=["TRUE:YES", "TRUE:NO"], columns=["PRED:YES", "PRED:NO"]))

Confusion Matrix (rows: true, columns: pred):


PRED:YES  PRED:NO
TRUE:YES        87        8
TRUE:NO         20      107

# 🧮 Evaluation Results: Confusion Matrix and Metrics

## 🧩 Step 1. Initial Confusion Matrix

| Actual \ Predicted | Pred 0 | Pred 1 |
|--------------------|--------|--------|
| **0** | 256 | 117 |
| **1** | 22  | 105 |

**Total predicted “1” = 117 + 105 = 222**

---

## 🧩 Step 2. Re-evaluated Predicted “1” Group

| Actual \ NewPred | New Pred 0 | New Pred 1 |
|------------------|-------------|-------------|
| **0** | 107 | 20 |
| **1** | 8   | 87 |

Interpretation:
- 107 (previously false positives) corrected to class 0  
- 20 remained false positives  
- 8 (false negatives in this group) corrected to class 0  
- 87 remain true positives  

---

## ✅ Step 3. Final Confusion Matrix

| Actual \ Predicted | Pred 0 | Pred 1 |
|--------------------|--------|--------|
| **0** | 256 + 107 = **363** | 20 |
| **1** | 22 + 8 = **30**     | 87 |

### ✅ Final Confusion Matrix

| Actual \ Predicted | Pred 0 | Pred 1 |
|--------------------|--------|--------|
| **0** | **363** | **20** |
| **1** | **30**  | **87** |

**Total samples = 500**

---

## 🧮 Step 4. Metrics

Let:
- **TP** = 87  
- **FP** = 20  
- **FN** = 30  
- **TN** = 363  

---

### **Class 1 (“valid”)**

\[
\text{Precision}_1 = \frac{TP}{TP + FP} = \frac{87}{107} = 0.813
\]

\[
\text{Recall}_1 = \frac{TP}{TP + FN} = \frac{87}{117} = 0.744
\]

\[
\text{F1}_1 = 2 \times \frac{0.813 \times 0.744}{0.813 + 0.744} = 0.777
\]

---

### **Class 0 (“invalid”)**

\[
\text{Precision}_0 = \frac{TN}{TN + FN} = \frac{363}{393} = 0.924
\]

\[
\text{Recall}_0 = \frac{TN}{TN + FP} = \frac{363}{383} = 0.948
\]

\[
\text{F1}_0 = 2 \times \frac{0.924 \times 0.948}{0.924 + 0.948} = 0.936
\]

---

## 📊 Final Results Summary

| Class | Precision | Recall | F1 |
|-------|------------|--------|----|
| **0 (Invalid)** | **0.924** | **0.948** | **0.936** |
| **1 (Valid)**   | **0.813** | **0.744** | **0.777** |

---

### ✅ Overall Accuracy

\[
\text{Accuracy} = \frac{TP + TN}{Total} = \frac{87 + 363}{500} = 0.900
\]

**Final Accuracy ≈ 90.0%**

---

🧠 **Key Insight:**  
The classifier performs very well on the **invalid** class, with strong precision and recall (F1 ≈ 0.94).  
However, **valid** edits show a slight drop in recall (0.74), suggesting that while precision remains strong, some valid comment–edit justifications are still being missed.  
Further improvements might focus on expanding positive examples or refining prompt alignment.
